In [ ]:
import torch
import sys
import gc

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, WrapperConfig, SMOL_LM_135M

In [3]:
cfg = WrapperConfig(
    model_id_or_path=SMOL_LM_135M,
    max_samples=1000,
    data_module_choice="conversation",
    notebook_mode=True,
    tuning_mode="sft",
    eval_data_mode="fixed",
)

wrapper = TrainerWrapper(cfg)

In [ ]:
wrapper.init_model()

In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [66]:
wrapper.init_data_module()

2025-01-01 20:59:56.902 | INFO     | model.utils:__init__:105 - Cache dir: ../dataset_caches/conversation_data_module
2025-01-01 20:59:56.904 | INFO     | model.utils:setup:123 - Loading dataset for stage fit
2025-01-01 20:59:56.904 | INFO     | model.utils:setup:125 - Processing dataset for stage fit, workers: 1, cache dir ../dataset_caches/conversation_data_module
2025-01-01 20:59:56.904 | INFO     | dataset.code:load_dataset:167 - Loading dataset
Parameter 'function'=<bound method ConversationDataModule.process_samples_batch_sft of <dataset.code.ConversationDataModule object at 0x305e84d10>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning 

In [ ]:
wrapper.data_module.load_dataset()

In [ ]:
# Tokenizer dev
tokenized = wrapper.data_module.tokenize_conversation([wrapper.data_module.train_dataset[0]["conversation"]])
for k in tokenized.keys():
    tokenized[k] = torch.as_tensor(tokenized[k], dtype=torch.long)
wrapper.data_module.visualize_sample(tokenized)

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.train()